In [9]:
from sdRDM.generator import generate_python_api


generate_python_api("../specifications/model.md", "../", "pyEED")

# PYEED data model

In [10]:
%reload_ext autoreload
%autoreload 2

from pyEED.core.proteinsequence import ProteinSequence
from pyEED.ncbi.utils import get_nucleotide_sequences

## Query NCBI

In [11]:
from pyEED.core.proteinsequence import ProteinSequence

tem1 = ProteinSequence.from_ncbi("QGC48744.1")

print(tem1)

ProteinSequence
├── id = QGC48744.1
├── name = TEM family beta-lactamase
├── sequence = ALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW
├── organism
│   └── Organism
│       ├── id = taxon:562
│       ├── name = Escherichia coli
│       └── taxonomy_id = 562
├── regions
│   └── 0
│       └── Region
│           ├── id = region0
│           ├── start = 0
│           ├── end = 278
│           ├── note = beta-lactamase TEM; Provisional
│           ├── name = PRK15442
│           └── cross_reference = CDD:185339
└── coding_sequence
    └── NucleotideSequence
        ├── id = MN158355.1
        ├── regions
        │   └── 0
        │       └── Region
        │           ├── id = MN158355.1
        │           ├── start = 1
        │           └── end = 837
        

## BLAST search

In [8]:
type(tem1.ec_number)

NoneType

In [3]:
blast_result_sequences = tem1.blast(n_hits=40)

Running blast search for TEM family beta-lactamase from Escherichia coli...


Fetching protein sequences: 100%|██████████| 40/40 [06:05<00:00,  9.15s/it]


In [ ]:
get_nucleotide_sequences(blast_result_sequences)

Fetching nucleotide sequences: 100%|██████████| 40/40 [06:18<00:00,  9.47s/it]


In [ ]:
print(blast_result_sequences[7])

ProteinSequence
├── id = KAH8247117.1
├── name = hypothetical protein
├── sequence = MTTYFNYPSKELQNELRGIAERIVAPGKGILAADESVSTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLSENISGVILFHETLYQKADDGTPFVDILKKKGIIPGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLGRPWALTFSYGRALQASVLRAWGGKKENIGAGQNELLKRAKANALACQGQYVAGSIPSYAANASLFVAQHKY
├── organism
│   └── Organism
│       ├── id = taxon:46829
│       ├── name = Drosophila birchii
│       └── taxonomy_id = 46829
├── regions
│   └── 0
│       └── Region
│           ├── id = region8
│           ├── start = 14
│           ├── end = 363
│           ├── note = Fructose-bisphosphate aldolase class-I; pfam00274
│           ├── name = Glycolytic
│           └── cross_reference = CDD:425574
├── sites
│   ├── 0
│   │   └── Site
│   │       ├── id = site24
│   │       ├── name = active
│   │       ├── type = active
│   │       ├── positi

## Storing `ProteinSequence`s in a database



In [12]:
from sdRDM import DataModel
from sdrdm_database import DBConnector, create_tables

### Setting up a local MySQL database

First, a local MySQL database needs to be setup. Therefore, we run a docker container with a MySQL database. 

>[!NOTE]
>
>If docker is not isntalled on your system, please follow the instructions on the [docker website](https://docs.docker.com/get-docker/).


In case this notebook is run on a macOS system with a M1 chip, the following command needs to be run in the terminal first:

>```bash
>export DOCKER_DEFAULT_PLATFORM=linux/amd64 
>```

Next, navigate to the directory where this notebook is located and run the following command to start the docker container:

>```bash
>docker compose up -d
>```

### Connect to the database

In [13]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

In [14]:
# Create the tables in the database
create_tables(db_connector=db, model=ProteinSequence)

Could not add primary key id for table ProteinSequence: 


OperationalError: (pymysql.err.OperationalError) (1033, "Incorrect information in file: './db/proteinsequence.frm'")
[SQL: ALTER TABLE ProteinSequence ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

### Populate the database

In [ ]:
# Transform the data into a list of dictionaries
blast_result_data = [result.dict() for result in blast_result_sequences]

# Insert the data into the database
db.connection.insert("Reactant", blast_result_data)

NameError: name 'blast_result_sequences' is not defined